### Instalacja

In [ ]:
%cd /content/
!git clone https://github.com/Bartolo72/phisher
%cd /content/phisher
!pip install -r requirements.txt -q
!pip install -e . -q
%cd /content/

### XAI

In [ ]:
from phisher.xai import GradCAM
from phisher.module import PhisherhModule
from phisher.model import PhisherEmbeddingModel
from phisher.dataset import PhishingEmbeddingDataset

In [ ]:
import torch

#### Load model

In [ ]:
model_ckpt_path: str = "checkpoints/embeddings_model.ckpt"

In [ ]:
model: PhisherEmbeddingModel = PhisherEmbeddingModel(vocab_size=84, embedding_dim=100, out_features=1)

In [ ]:
PhisherhModule.load_from_checkpoint(model_ckpt_path, model=model)

#### Visualize Grad-CAM

In [ ]:
def visualize_url_with_cam(url: str, cam: torch.Tensor) -> str:
    cam = cam.mean(dim=1).squeeze()
    cam = cam.cpu().detach().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min())
    url = url[: len(cam)]

    colored_url = ""
    for char, intensity in zip(url, cam):
        color_intensity = int(
            255 * (1 - intensity)
        )  # Scale intensity (higher = darker)
        hex_color = f"{color_intensity:02x}ff{color_intensity:02x}"
        colored_url += f'<span style="background-color: #{hex_color};">{char}</span>'

    return colored_url

In [ ]:
grad_cam = GradCAM(model, target_layer=model.conv2)

In [ ]:
target_class: torch.Tensor = torch.tensor([0]) # 0 phishing, 1 safe URL
target_url: str = "aeinpost.me"

In [ ]:
placeholder_dataset = PhishingEmbeddingDataset(csv_file_path="")

In [ ]:
input_tensor: torch.Tensor
input_tensor = placeholder_dataset.parse_url(target_url)
input_tensor = placeholder_dataset.pad_or_trim(input_tensor)

In [ ]:
cam = grad_cam.generate_cam(input_tensor, target_class)
visualized_url: str = visualize_url_with_cam(url=target_url, cam=cam)

In [ ]:
from IPython.core.display import display, HTML
display(HTML(visualized_url))